**importing the required libraries**

In [9]:
import cv2
import dlib
import face_recognition
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras  # Accessing Keras through TensorFlow
from tensorflow.keras.models import model_from_json 
from tensorflow.keras.preprocessing import image

**loading the image to detect**

In [10]:
image_to_detect = cv2.imread(f'Images\Modi-Meloni-G20-Summit-Brazil.jpg')

**Load the model and load the weights**

In [11]:
face_exp_model = model_from_json(open("dataset\\facial_expression_model_structure.json","r").read())
face_exp_model.load_weights('dataset\\facial_expression_model_weights.h5')

**declare the emotions label**

In [12]:
emotions_label = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

**detect all faces in the image**

**arguments are image, no_of_times_to_upsample, model**

In [13]:
all_face_locations = face_recognition.face_locations(image_to_detect,model='hog')

In [14]:
# print the number of faces detected

print('There are {} no of faces in this image'.format(len(all_face_locations)))

There are 2 no of faces in this image


In [15]:
output_dir = 'detected_faces'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [16]:
# looping through the face locations

for index, current_face_location in enumerate(all_face_locations):
    # splitting the tuple to get the four position values of current face
    top_pos, right_pos, bottom_pos, left_pos = current_face_location
    # printing the location of current face
    print('Found face {} at top:{}, right:{}, bottom:{}, left:{}'.format(index + 1, top_pos, right_pos, bottom_pos, left_pos))
    # slicing the current face from main image
    current_face_image = image_to_detect[top_pos:bottom_pos, left_pos:right_pos]
    # draw rectangle around the face detected
    cv2.rectangle(image_to_detect, (left_pos, top_pos), (right_pos, bottom_pos), (0, 0, 255), 2)

    # preprocess input, convert it to an image like as the data in dataset
    # convert to grayscale
    current_face_image = cv2.cvtColor(current_face_image, cv2.COLOR_BGR2GRAY)
    # resize to 48x48 px size
    current_face_image = cv2.resize(current_face_image, (48, 48))
    # convert the PIL image into a 3d numpy array
    img_pixels = image.img_to_array(current_face_image)
    # expand the shape of an array into single row multiple columns
    img_pixels = np.expand_dims(img_pixels, axis=0)
    # pixels are in range of [0, 255]. normalize all pixels in scale of [0, 1]
    img_pixels /= 255

    # do prodiction using model, get the prediction values for all 7 expressions
    exp_predictions = face_exp_model.predict(img_pixels)
    # find max indexed prediction value (0 till 7)
    max_index = np.argmax(exp_predictions[0])
    # get corresponding lable from emotions_label
    emotion_label = emotions_label[max_index]

    # display the name as text in the image
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(image_to_detect, emotion_label, (left_pos, bottom_pos), font, 0.5, (255, 255, 255), 1)

# showing the current face with rectangle drawn
cv2.imshow("Face emotions", image_to_detect)

# wait for a key press and then close all windows
cv2.waitKey(0)  # Waits indefinitely until a key is pressed
cv2.destroyAllWindows()  # Closes all open windows

Found face 1 at top:87, right:218, bottom:149, left:156
1/1 [==============================] - 0s 85ms/step
Found face 2 at top:38, right:403, bottom:113, left:328
1/1 [==============================] - 0s 18ms/step
